In [2]:
!unzip ../kfood.zip -d ../BP_OB_Model/img_zip

Archive:  ../kfood.zip
 extracting: ../BP_OB_Model/img_zip/구이.zip  
 extracting: ../BP_OB_Model/img_zip/국.zip  
 extracting: ../BP_OB_Model/img_zip/기타.zip  
 extracting: ../BP_OB_Model/img_zip/김치.zip  
 extracting: ../BP_OB_Model/img_zip/나물.zip  
 extracting: ../BP_OB_Model/img_zip/떡.zip  
 extracting: ../BP_OB_Model/img_zip/만두.zip  
 extracting: ../BP_OB_Model/img_zip/면.zip  
 extracting: ../BP_OB_Model/img_zip/무침.zip  
 extracting: ../BP_OB_Model/img_zip/밥.zip  
 extracting: ../BP_OB_Model/img_zip/볶음.zip  
 extracting: ../BP_OB_Model/img_zip/쌈.zip  
 extracting: ../BP_OB_Model/img_zip/음청류.zip  
 extracting: ../BP_OB_Model/img_zip/장.zip  
 extracting: ../BP_OB_Model/img_zip/장아찌.zip  
 extracting: ../BP_OB_Model/img_zip/적.zip  
 extracting: ../BP_OB_Model/img_zip/전.zip  
 extracting: ../BP_OB_Model/img_zip/전골.zip  
 extracting: ../BP_OB_Model/img_zip/조림.zip  
 extracting: ../BP_OB_Model/img_zip/죽.zip  
 extracting: ../BP_OB_Model/img_zip/찌개.zip  
 extracting: ../BP_OB_Model/img_zip/찜.z

In [3]:
import os
import zipfile

# Train_Test 디렉토리의 경로 설정
directory = '../BP_OB_Model/img_zip/'

# 해당 디렉토리 내의 모든 파일을 순회
for filename in os.listdir(directory):
    if filename.endswith('.zip'):
        # 압축 파일의 전체 경로
        zip_path = os.path.join(directory, filename)

        # Zip 파일 열기
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            # 압축 해제
            zip_ref.extractall("../BP_OB_Model/img/")
            print(f'{filename} has been extracted.')

장.zip has been extracted.
김치.zip has been extracted.
국.zip has been extracted.
떡.zip has been extracted.
만두.zip has been extracted.
튀김.zip has been extracted.
전골.zip has been extracted.
쌈.zip has been extracted.
나물.zip has been extracted.
죽.zip has been extracted.
장아찌.zip has been extracted.
무침.zip has been extracted.
구이.zip has been extracted.
탕.zip has been extracted.
볶음.zip has been extracted.
찜.zip has been extracted.
면.zip has been extracted.
음청류.zip has been extracted.
밥.zip has been extracted.
한과.zip has been extracted.
기타.zip has been extracted.
회.zip has been extracted.
찌개.zip has been extracted.
조림.zip has been extracted.
전.zip has been extracted.
적.zip has been extracted.
해물.zip has been extracted.


## 이미지 분류 정리 및 테그

In [4]:
# Test Cell
from PIL import Image
import os
import glob
import shutil

def get_subdirectories(path):
    # 주어진 경로의 모든 하위 폴더를 반환합니다.
    subdirectories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return subdirectories

def convert_to_yolo_format(label_file, image_folder, destination_folder):
    error,duplicate = 0,0
    # 이미지 폴더 내의 모든 파일 불러오기
    all_files = {os.path.splitext(f)[0]: f for f in glob.glob(image_folder + '/*')}
    
    # .properties, .txt, .csv 파일 제외
    valid_files = {k: v for k, v in all_files.items() if not v.endswith(('.properties', '.txt', '.csv'))}

    # 라벨 파일 읽기
    with open(label_file, 'r') as file:
        lines = file.readlines()

    for line in lines:
        # 라벨 데이터 파싱
        image_name, coords = line.split('=')
        coord_parts = coords.split(',')

        # Check if all four coordinates are present
        if len(coord_parts) != 4 or '' in coord_parts:
            print(f"Skipping incomplete data for image {image_name}")
            error += 1
            continue
    
        x_min, y_min, Width, Height = map(int, coords.split(','))
        _, food_code, _ = image_name.split("_")

        # 해당 이미지 파일 찾기
        image_path = valid_files.get(image_folder + f'/{image_name}')

        if image_path:
            # Check if the image file exists
            if not os.path.exists(image_path):
                print(f"Image file not found: {image_path}, skipping...")
                duplicate += 1
                continue
            # 이미지 크기 가져오기
            with Image.open(image_path) as img:
                W, H = img.size

            # 상대 좌표로 변환
            x_center = (x_min + x_min + Width) / 2.0 / W
            y_center = (y_min + y_min + Height) / 2.0 / H
            width = ((x_min + Width) - x_min) / W
            height = ((y_min + Height) - y_min) / H

            # YOLO 형식으로 저장
            yolo_format = f"{int(food_code)} {x_center} {y_center} {width} {height}\n"
            txt_file_path = os.path.splitext(image_path)[0] + '.txt'
            with open(txt_file_path, 'w') as yolo_file:
                yolo_file.write(yolo_format)

            # 이미지와 .txt 파일을 목적지 폴더로 이동
            # (Train_img 와 Train_label을 분류하는 이유 : 분류 확인을 위해서 진행합니다 만약 확인이 불필요하면 두개다 Train_img로 이동하면 된다.)
            shutil.move(image_path, os.path.join(destination_folder, os.path.basename(image_path)))
            shutil.move(txt_file_path, os.path.join(destination_folder, os.path.basename(txt_file_path)))

data_path = "../BP_OB_Model/img"
destination_folder = "/data/BP/BP_OB_Model/Train/"

main_category = get_subdirectories(data_path)
for mc in main_category:
    sub_category = get_subdirectories(f'{data_path}/{mc}')
    for sb in sub_category:
        convert_to_yolo_format(f'{data_path}/{mc}/{sb}/crop_area.properties', f'{data_path}/{mc}/{sb}', destination_folder)


Skipping incomplete data for image Img_122_0108
Skipping incomplete data for image Img_122_0998
Skipping incomplete data for image Img_123_0266
Skipping incomplete data for image Img_119_0028
Skipping incomplete data for image Img_119_0402
Skipping incomplete data for image Img_024_0175
Skipping incomplete data for image Img_024_0242
Skipping incomplete data for image Img_023_0162
Skipping incomplete data for image Img_023_0194
Skipping incomplete data for image Img_023_0227
Skipping incomplete data for image Img_023_0488
Skipping incomplete data for image Img_023_0982
Skipping incomplete data for image Img_028_0992
Skipping incomplete data for image Img_047_0060
Skipping incomplete data for image Img_047_0263
Skipping incomplete data for image Img_047_0294
Skipping incomplete data for image Img_047_0861
Skipping incomplete data for image Img_046_0116
Skipping incomplete data for image Img_046_0194
Skipping incomplete data for image Img_046_0998
Skipping incomplete data for image Img_0

In [8]:
# 학습 데이터 갯수 확인
import os

def count_files_in_folder(folder_path):
    # Counts the number of files in a given folder.
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return 0

    all_items = os.listdir(folder_path)
    files = [item for item in all_items if os.path.isfile(os.path.join(folder_path, item))]
    return len(files) / 2.0

# Example usage
file_count = count_files_in_folder('../BP_OB_Model/Train')
print(f"There are {file_count} files in the folder '{'../BP_OB_Model/Train'}'.")


There are 51320.0 files in the folder '../BP_OB_Model/Train'.


In [10]:
import os
from collections import defaultdict

def count_images_by_food_code(folder_path):
    # Count images by food code in the given folder.
    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return {}

    image_count = defaultdict(int)
    
    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)) and file.startswith("Img_"):
            parts = file.split('_')
            if len(parts) >= 3:
                food_code = parts[1]
                image_count[food_code] += 1

    return image_count

folder_path = '../BP_OB_Model/Train'
image_counts = count_images_by_food_code(folder_path)

for food_code, count in image_counts.items():
    print(f"음식코드 {food_code}: {count / 2.0}개의 이미지")


음식코드 052: 502.0개의 이미지
음식코드 069: 385.0개의 이미지
음식코드 102: 908.0개의 이미지
음식코드 044: 845.0개의 이미지
음식코드 076: 807.0개의 이미지
음식코드 080: 946.0개의 이미지
음식코드 142: 272.0개의 이미지
음식코드 149: 770.0개의 이미지
음식코드 055: 379.0개의 이미지
음식코드 060: 360.0개의 이미지
음식코드 117: 702.0개의 이미지
음식코드 043: 767.0개의 이미지
음식코드 081: 829.0개의 이미지
음식코드 105: 706.0개의 이미지
음식코드 074: 748.0개의 이미지
음식코드 097: 213.0개의 이미지
음식코드 073: 816.0개의 이미지
음식코드 059: 558.0개의 이미지
음식코드 090: 191.0개의 이미지
음식코드 026: 297.0개의 이미지
음식코드 116: 156.0개의 이미지
음식코드 045: 941.0개의 이미지
음식코드 063: 260.0개의 이미지
음식코드 082: 699.0개의 이미지
음식코드 099: 565.0개의 이미지
음식코드 103: 670.0개의 이미지
음식코드 051: 698.0개의 이미지
음식코드 077: 886.0개의 이미지
음식코드 048: 621.0개의 이미지
음식코드 032: 10.0개의 이미지
음식코드 134: 303.0개의 이미지
음식코드 147: 433.0개의 이미지
음식코드 049: 832.0개의 이미지
음식코드 106: 10.0개의 이미지
음식코드 104: 771.0개의 이미지
음식코드 070: 602.0개의 이미지
음식코드 078: 900.0개의 이미지
음식코드 000: 224.0개의 이미지
음식코드 019: 215.0개의 이미지
음식코드 009: 124.0개의 이미지
음식코드 028: 39.0개의 이미지
음식코드 065: 662.0개의 이미지
음식코드 087: 872.0개의 이미지
음식코드 113: 157.0개의 이미지
음식코드 002: 215.0개의 이미지
음식코드 135: 219

In [12]:
import os
import shutil
import random
from collections import defaultdict

def move_images_and_labels(source_path, dest_path, food_code, number_to_move):
    """ Move a specified number of images and labels for a given food code. """
    images = [img for img in os.listdir(source_path) if img.startswith(f"Img_{food_code}_") and os.path.splitext(img)[1].lower() != '.txt']
    selected_images = random.sample(images, min(number_to_move, len(images)))

    # for img in selected_images:
    #     shutil.move(os.path.join(source_img_path, img), os.path.join(dest_img_path, img))
    #     label_file = img.replace('.*', '.txt')
    #     shutil.move(os.path.join(source_label_path, label_file), os.path.join(dest_label_path, label_file))
    
    for img in selected_images:
        shutil.move(os.path.join(source_path, img), os.path.join(dest_path, img))
        # Extract the base name without extension and add '.txt'
        base_name = os.path.splitext(img)[0]
        label_file = base_name + '.txt'
        shutil.move(os.path.join(source_path, label_file), os.path.join(dest_path, label_file))

    return len(selected_images)

def main(train_path, val_path): 
    image_counts = count_images_by_food_code(train_path)
    total_moved = 0

    for food_code, count in image_counts.items():
        number_to_move = int(count * 0.2)  # 20% of images
        moved = move_images_and_labels(train_path, val_path, food_code, number_to_move)
        total_moved += moved
        print(f"Moved {moved} images and labels for 음식코드 {food_code}")

    print(f"Total moved: {total_moved} images and labels")

# Paths
train_path = '../BP_OB_Model/Train'
val_path = '../BP_OB_Model/Val'

# Execute
main(train_path, val_path)

Moved 200 images and labels for 음식코드 052
Moved 154 images and labels for 음식코드 069
Moved 363 images and labels for 음식코드 102
Moved 338 images and labels for 음식코드 044
Moved 322 images and labels for 음식코드 076
Moved 378 images and labels for 음식코드 080
Moved 108 images and labels for 음식코드 142
Moved 308 images and labels for 음식코드 149
Moved 151 images and labels for 음식코드 055
Moved 144 images and labels for 음식코드 060
Moved 280 images and labels for 음식코드 117
Moved 306 images and labels for 음식코드 043
Moved 331 images and labels for 음식코드 081
Moved 282 images and labels for 음식코드 105
Moved 299 images and labels for 음식코드 074
Moved 85 images and labels for 음식코드 097
Moved 326 images and labels for 음식코드 073
Moved 223 images and labels for 음식코드 059
Moved 76 images and labels for 음식코드 090
Moved 118 images and labels for 음식코드 026
Moved 62 images and labels for 음식코드 116
Moved 376 images and labels for 음식코드 045
Moved 104 images and labels for 음식코드 063
Moved 279 images and labels for 음식코드 082
Moved 226 images an

In [31]:
def get_subdirectories(path):
    # 주어진 경로의 모든 하위 폴더를 반환합니다.
    subdirectories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return subdirectories

## Yolov8x

In [36]:
for mc in sorted(get_subdirectories("../BP_OB_Model/img/")):
    print(sorted(get_subdirectories(f"../BP_OB_Model/img/{mc}/")))
    print('='*100)
    

['갈비구이', '갈치구이', '고등어구이', '곱창구이', '닭갈비', '더덕구이', '떡갈비', '불고기', '삼겹살', '장어구이', '조개구이', '조기구이', '황태구이', '훈제오리']
['계란국', '떡국_만두국', '무국', '미역국', '북엇국', '시래기국', '육개장', '콩나물국']
['과메기', '양념치킨', '젓갈', '콩자반', '편육', '피자', '후라이드치킨']
['갓김치', '깍두기', '나박김치', '무생채', '배추김치', '백김치', '부추김치', '열무김치', '오이소박이', '총각김치', '파김치']
['가지볶음', '고사리나물', '미역줄기볶음', '숙주나물', '시금치나물', '애호박볶음']
['경단', '꿀떡', '송편']
['만두']
['라면', '막국수', '물냉면', '비빔냉면', '수제비', '열무국수', '잔치국수', '짜장면', '짬뽕', '쫄면', '칼국수', '콩국수']
['꽈리고추무침', '도라지무침', '도토리묵', '잡채', '콩나물무침', '홍어무침', '회무침']
['김밥', '김치볶음밥', '누룽지', '비빔밥', '새우볶음밥', '알밥', '유부초밥', '잡곡밥', '주먹밥']
['감자채볶음', '건새우볶음', '고추장진미채볶음', '두부김치', '떡볶이', '라볶이', '멸치볶음', '소세지볶음', '어묵볶음', '오징어채볶음', '제육볶음', '주꾸미볶음']
['보쌈']
['수정과', '식혜']
['간장게장', '양념게장']
['깻잎장아찌']
['떡꼬치']
['감자전', '계란말이', '계란후라이', '김치전', '동그랑땡', '생선전', '파전', '호박전']
['곱창전골']
['갈치조림', '감자조림', '고등어조림', '꽁치조림', '두부조림', '땅콩조림', '메추리알장조림', '연근조림', '우엉조림', '장조림', '코다리조림']
['전복죽', '호박죽']
['김치찌개', '닭계장', '동태찌개', '된장찌개', '순두부찌개']
['갈비찜', '계란찜', '김치찜', '꼬

In [37]:
# yaml 파일 생성
import yaml

data ={'train' : "../Train/Train_img/",
       'val' : "../Val/Val_img/",
       'names' : ['갈비구이', '갈치구이', '고등어구이', '곱창구이', '닭갈비', '더덕구이', '떡갈비', '불고기', '삼겹살', '장어구이', '조개구이', '조기구이', '황태구이', '훈제오리',
                 '계란국', '떡국_만두국', '무국', '미역국', '북엇국', '시래기국', '육개장', '콩나물국',
                 '과메기', '양념치킨', '젓갈', '콩자반', '편육', '피자', '후라이드치킨',
                 '갓김치', '깍두기', '나박김치', '무생채', '배추김치', '백김치', '부추김치', '열무김치', '오이소박이', '총각김치', '파김치',
                 '가지볶음', '고사리나물', '미역줄기볶음', '숙주나물', '시금치나물', '애호박볶음',
                 '경단', '꿀떡', '송편',
                 '만두',
                 '라면', '막국수', '물냉면', '비빔냉면', '수제비', '열무국수', '잔치국수', '짜장면', '짬뽕', '쫄면', '칼국수', '콩국수',
                 '꽈리고추무침', '도라지무침', '도토리묵', '잡채', '콩나물무침', '홍어무침', '회무침',
                 '김밥', '김치볶음밥', '누룽지', '비빔밥', '새우볶음밥', '알밥', '유부초밥', '잡곡밥', '주먹밥',
                 '감자채볶음', '건새우볶음', '고추장진미채볶음', '두부김치', '떡볶이', '라볶이', '멸치볶음', '소세지볶음', '어묵볶음', '오징어채볶음', '제육볶음', '주꾸미볶음',
                 '보쌈',
                 '수정과', '식혜',
                 '간장게장', '양념게장',
                 '깻잎장아찌',
                 '떡꼬치',
                 '감자전', '계란말이', '계란후라이', '김치전', '동그랑땡', '생선전', '파전', '호박전',
                 '곱창전골',
                 '갈치조림', '감자조림', '고등어조림', '꽁치조림', '두부조림', '땅콩조림', '메추리알장조림', '연근조림', '우엉조림', '장조림', '코다리조림',
                 '전복죽', '호박죽',
                 '김치찌개', '닭계장', '동태찌개', '된장찌개', '순두부찌개',
                 '갈비찜', '계란찜', '김치찜', '꼬막찜', '닭볶음탕', '수육', '순대', '족발', '찜닭', '해물찜',
                 '갈비탕', '감자탕', '곰탕_설렁탕', '매운탕', '삼계탕', '추어탕',
                 '고추튀김', '새우튀김', '오징어튀김',
                 '약과', '약식', '한과',
                 '멍게', '산낙지',
                 '물회', '육회'],
       'nc' : 150}

with open('../BP_OB_Model/Food_OD.yaml', 'w') as f:
       yaml.dump(data, f)

with open('../BP_OB_Model/Food_OD.yaml', 'r') as f:
       Food_OD_yaml = yaml.safe_load(f)
       display(Food_OD_yaml)

{'name': ['갈비구이',
  '갈치구이',
  '고등어구이',
  '곱창구이',
  '닭갈비',
  '더덕구이',
  '떡갈비',
  '불고기',
  '삼겹살',
  '장어구이',
  '조개구이',
  '조기구이',
  '황태구이',
  '훈제오리',
  '계란국',
  '떡국_만두국',
  '무국',
  '미역국',
  '북엇국',
  '시래기국',
  '육개장',
  '콩나물국',
  '과메기',
  '양념치킨',
  '젓갈',
  '콩자반',
  '편육',
  '피자',
  '후라이드치킨',
  '갓김치',
  '깍두기',
  '나박김치',
  '무생채',
  '배추김치',
  '백김치',
  '부추김치',
  '열무김치',
  '오이소박이',
  '총각김치',
  '파김치',
  '가지볶음',
  '고사리나물',
  '미역줄기볶음',
  '숙주나물',
  '시금치나물',
  '애호박볶음',
  '경단',
  '꿀떡',
  '송편',
  '만두',
  '라면',
  '막국수',
  '물냉면',
  '비빔냉면',
  '수제비',
  '열무국수',
  '잔치국수',
  '짜장면',
  '짬뽕',
  '쫄면',
  '칼국수',
  '콩국수',
  '꽈리고추무침',
  '도라지무침',
  '도토리묵',
  '잡채',
  '콩나물무침',
  '홍어무침',
  '회무침',
  '김밥',
  '김치볶음밥',
  '누룽지',
  '비빔밥',
  '새우볶음밥',
  '알밥',
  '유부초밥',
  '잡곡밥',
  '주먹밥',
  '감자채볶음',
  '건새우볶음',
  '고추장진미채볶음',
  '두부김치',
  '떡볶이',
  '라볶이',
  '멸치볶음',
  '소세지볶음',
  '어묵볶음',
  '오징어채볶음',
  '제육볶음',
  '주꾸미볶음',
  '보쌈',
  '수정과',
  '식혜',
  '간장게장',
  '양념게장',
  '깻잎장아찌',
  '떡꼬치',
  '감자전',
  '계란말이',
  '계란후라이',
  '김치전',
  '동그랑땡',
  '생선전'

In [39]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.2/661.2 kB 8.9 MB/s eta 0:00:00:00:0100:01


In [1]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt')
model.to('cuda')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_

In [2]:
import mlflow
import re,os
# Databricks MLflow 추적 서버 URI 설정
databricks_uri = "databricks"  # Databricks 워크스페이스의 MLflow URI
# mlflow.set_tracking_uri(databricks_uri)
os.environ['MLFLOW_TRACKING_URI'] = databricks_uri
# # Set the experiment path
mlflow.set_experiment("/Users/20182562@oasis.inje.ac.kr/Yolov")

def on_fit_epoch_end(trainer):
    if mlflow:
        metrics_dict = {f"{re.sub('[()]', '', k)}": float(v) for k, v in trainer.metrics.items()}
        mlflow.log_metrics(metrics=metrics_dict, step=trainer.epoch)

# 실험 세션 생성
try: 
     with mlflow.start_run():
          model.add_callback("on_fit_epoch_end",on_fit_epoch_end)
          result = model.train(data='../BP_OB_Model/Food_OD.yaml', epochs=100, patience=30, batch=32, imgsz=416)
          # Log the model to MLflow
          mlflow.pytorch.log_model(model, "model")

	
except mlflow.exceptions.MlflowException as e:
    print(f"An Mlflow error occurred: {str(e)}")

New https://pypi.org/project/ultralytics/8.0.231 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=../BP_OB_Model/Food_OD.yaml, epochs=100, time=None, patience=30, batch=32, imgsz=416, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /data/BP/BP_OB_Model/Train.cache... 41071 images, 0 backgrounds, 33 corrupt: 100%|██████████| 41102/41102 [00:00<?, ?it/s]

train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_000_0233.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0181.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0247.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0286.jpg: ignoring corrupt image/label: invalid image format GIF
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0311.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_001_0355.jpg: ignoring corrupt image/label: invalid image format GIF
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_002_0586.JPG: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_002_0879.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_004_0412.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /data/BP/BP_OB_Model/Train/Img_004_0419.jpg: corrupt JPEG restored and saved
trai


val: Scanning /data/BP/BP_OB_Model/Val.cache... 10188 images, 0 backgrounds, 10 corrupt: 100%|██████████| 10198/10198 [00:00<?, ?it/s]

val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_000_0097.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_001_0362.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_006_0293.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_007_0568.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_007_0580.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_016_0993.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_017_0415.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_018_0405.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_019_0200.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_019_0339.jpg: ignoring corrupt image/label: invalid image format GIF
val: WARNING ⚠️ /data/BP/BP_OB_Model/Val/Img_023_0079.jpg: corrupt 

...

                   all      10188      10188      0.725      0.678      0.716      0.509

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     65/100      6.31G     0.8693     0.8619      1.189         92        416:   2%|▏         | 21/1284 [00:07<07:14,  2.91it/s]libpng warning: iCCP: known incorrect sRGB profile
     65/100      6.31G     0.8544       0.82      1.176         92        416:   5%|▍         | 64/1284 [00:22<07:02,  2.88it/s]libpng warning: iCCP: known incorrect sRGB profile
     65/100      6.31G     0.8616     0.8159      1.181         79        416:   9%|▉         | 118/1284 [00:40<06:43,  2.89it/s]libpng warning: iCCP: known incorrect sRGB profile
     65/100      6.31G     0.8604     0.8161       1.18         75        416:  10%|█         | 129/1284 [00:44<06:38,  2.90it/s]libpng warning: iCCP: known incorrect sRGB profile
     65/100      6.31G     0.8594     0.8132      1.184         80        416:  18%|█▊        | 225/1284 [01:17<06:04,  2.91it/s]libpng warning: iCCP: known incorrect sRGB profile
     65/100      6.31G     0.8589     0.8143      1.184         91        416:  19%|█▉        | 244/12

                   all      10188      10188      0.727      0.678      0.716      0.509

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     66/100      6.31G     0.8669     0.8205      1.191         76        416:   1%|          | 8/1284 [00:02<07:21,  2.89it/s]libpng warning: iCCP: known incorrect sRGB profile
     66/100      6.31G     0.8645     0.8205      1.185         83        416:   4%|▍         | 57/1284 [00:19<07:00,  2.92it/s]libpng warning: iCCP: known incorrect sRGB profile
     66/100      6.31G     0.8627     0.8176      1.185         82        416:   5%|▌         | 70/1284 [00:24<06:58,  2.90it/s]libpng warning: iCCP: known incorrect sRGB profile
     66/100      6.31G     0.8655      0.822      1.186         69        416:  14%|█▎        | 176/1284 [01:00<06:21,  2.90it/s]libpng warning: iCCP: known incorrect sRGB profile
     66/100      6.31G     0.8603      0.823      1.181         89        416:  20%|██        | 258/1284 [01:28<05:55,  2.89it/s]libpng warning: iCCP: known incorrect sRGB profile
     66/100      6.31G     0.8581     0.8262      1.183         93        416:  26%|██▌       | 330/1284

KeyboardInterrupt: 

In [13]:
# 
import requests
import os
import json
# kt-api lib
from datetime import datetime
import hmac, hashlib
from pytz import timezone
from PIL import Image
# naver-api lib
import uuid
import time
#yolov8
from ultralytics import YOLO

In [14]:
def img_size_pillow(img_path):
    image = Image.open(img_path)
    # print("Before :",image.size)  # 이미지의 크기 출력 (너비, 높이)
    if image.size[0] > image.size[1]:
        resized_image = image.resize((1440, 1440))
        resized_image.save(img_path)
    elif image.size[1] > image.size[0]:
        resized_image = image.resize((1440, 1440))
        resized_image.save(img_path)
    else:
        resized_image = image.resize((1440, 1440))
        resized_image.save(img_path)

In [26]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

model = YOLO("../BP_OB_Model/runs/detect/train9/weights/best.pt")
img_size_pillow('/data/BP/BP_Api_File/food/test17.jpg')
result = model.predict('/data/BP/BP_Api_File/food/test17.jpg')

# result[0].boxes에서 각 객체의 신뢰도를 확인하고, 가장 높은 신뢰도를 가진 객체 찾기
boxes = result[0].boxes
for box in boxes:
    if len(boxes.conf) > 0:
        # 신뢰도가 가장 높은 객체를 찾음
        max_conf_index = box.conf.argmax()
        highest_confidence = box.conf[max_conf_index].item()
        class_id = box.cls[max_conf_index].item()
        
        # 클래스 ID를 사용하여 음식 이름 찾기
        food_name = result[0].names[class_id]

        print(f"가장 높은 신뢰도를 가진 음식: {food_name}, 신뢰도: {highest_confidence:.2f}")
    else:
        print("탐지된 객체가 없습니다.")



# print(result[1])
# res_plotted = result[0].plot()

# plt.imshow(cv2.cvtColor(res_plotted,cv2.COLOR_BGR2RGB))

# plt.show()



image 1/1 /data/BP/BP_Api_File/food/test17.jpg: 416x416 2 비빔밥s, 7.8ms
Speed: 0.7ms preprocess, 7.8ms inference, 0.8ms postprocess per image at shape (1, 3, 416, 416)
가장 높은 신뢰도를 가진 음식: 비빔밥, 신뢰도: 0.84
가장 높은 신뢰도를 가진 음식: 비빔밥, 신뢰도: 0.25


## 불필요한 코드 셀

In [18]:
from PIL import Image
import os
import glob, shutil

def get_subdirectories(path):
    """ 주어진 경로의 모든 하위 폴더를 반환합니다. """
    subdirectories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
    return subdirectories

def convert_to_yolo_format(label_file, image_folder):
    # 이미지 폴더 내의 모든 파일 불러오기
    all_files = {os.path.splitext(f)[0]: f for f in glob.glob(image_folder + '/*')}
    
    # .properties, .txt, .csv 파일 제외
    valid_files = {k: v for k, v in all_files.items() if not v.endswith(('.properties', '.txt', '.csv'))}

    # 라벨 파일 읽기
    with open(label_file, 'r') as file:
        lines = file.readlines()

    for line in lines:
        # 라벨 데이터 파싱
        image_name, coords = line.split('=')
        x_min, y_min, x_max, y_max = map(int, coords.split(','))
        _, food_code, _ = image_name.split("_")

        # 해당 이미지 파일 찾기
        image_path = valid_files.get(image_folder+f'/{image_name}')

        if image_path:
            # Check if the image file exists
            if not os.path.exists(image_path):
                print(f"Image file not found: {image_path}, skipping...")
                continue
            # 이미지 크기 가져오기
            with Image.open(image_path) as img:
                W, H = img.size

            # 상대 좌표로 변환
            x_center = (x_min + x_max) / 2.0 / W
            y_center = (y_min + y_max) / 2.0 / H
            width = (x_max - x_min) / W
            height = (y_max - y_min) / H

            # YOLO 형식으로 저장
            yolo_format = f"{int(food_code)} {x_center} {y_center} {width} {height}\n"
            with open(os.path.splitext(image_path)[0] + '.txt', 'w') as yolo_file:
                yolo_file.write(yolo_format)
            break

data_path = "../BP_OB_Model/img"
# convert_to_yolo_format('../BP_OB_Model/img/구이/갈비구이/crop_area.properties', '../BP_OB_Model/img/구이/갈비구이')
main_category = get_subdirectories(data_path)
for mc in main_category:
    sub_category = get_subdirectories(f'{data_path}/{mc}')
    for sb in sub_category:
        convert_to_yolo_format(f'{data_path}/{mc}/{sb}/crop_area.properties', f'{data_path}/{mc}/{sb}/')


In [2]:
import shutil
import os

# Train 데이터셋의 라벨 파일 이동
train_labels = os.listdir('../BP_OB_Model/Train/Train_label')
for file in train_labels:
    shutil.move(f'../BP_OB_Model/Train/Train_label/{file}', '../BP_OB_Model/Train/Train_img/')

# Val 데이터셋의 라벨 파일 이동
val_labels = os.listdir('../BP_OB_Model/Val/Val_label')
for file in val_labels:
    shutil.move(f'../BP_OB_Model/Val/Val_label/{file}', '../BP_OB_Model/Val/Val_img/')